### Understanding the most efficient way to generate the next possible moves in a game of Reversi/Othello using Bitboards and Bitwise Operations

Date : 2024-03-04
Author : Malchemis


In [9]:
# Constants and Variables
size = 8
white_pieces = 0
black_pieces = 0

def set_state(bitboard: int, x: int, y: int, size: int):
    """Add a bit to the board by shifting a 1 to the left by x * size + y (flattened coordinates)"""
    return bitboard | (1 << (x * size + y))


def cell_count(bitboard: int):
    """Count the number of cells in the board"""
    return bitboard.bit_count()


def bits(number):
    """Generator to get the bits of a number"""
    bit = 1
    while number >= bit:
        if number & bit:
            yield bit
        bit <<= 1


def get_state(bitboard: int, x: int, y: int, size: int):
    """Return the state of the cell by shifting the board to the right by x * size + y and
    taking the least significant bit"""
    return (bitboard >> (x * size + y)) & 1


def get_indexes_move(move: int, size: int):
    """Return i and j indexes of the bitboard for a possible move"""
    position = move.bit_length() - 1
    # Calculate row and column indexes from the position
    i = position // size
    j = position % size
    return i, j


def print_pieces(bitboard: int, size: int):
    """Print the bit values of the board as a matrix of 0 and 1"""
    for i in range(size):
        for j in range(size):
            print(get_state(bitboard, i, j, size), end=' ')
        print()
    print()


def print_board(white: int, black: int, size: int):
    """Print the board with W for white, B for black and . for empty cells"""
    for i in range(size):
        for j in range(size):
            if get_state(white, i, j, size):
                print('W', end=' ')
            elif get_state(black, i, j, size):
                print('B', end=' ')
            else:
                print('.', end=' ')
        print()
    print()

# init the pieces
white_pieces = set_state(white_pieces, 3, 3, size)
white_pieces = set_state(white_pieces, 4, 4, size)
black_pieces = set_state(black_pieces, 3, 4, size)
black_pieces = set_state(black_pieces, 4, 3, size)

In [10]:
print("White Pieces")
print_pieces(white_pieces, size)
print("Black Pieces")
print_pieces(black_pieces, size)
print("Board")
print_board(white_pieces, black_pieces, size)

White Pieces
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 
0 0 0 0 1 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 

Black Pieces
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 1 0 0 0 
0 0 0 1 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 

Board
. . . . . . . . 
. . . . . . . . 
. . . . . . . . 
. . . W B . . . 
. . . B W . . . 
. . . . . . . . 
. . . . . . . . 
. . . . . . . . 


In [11]:
# in binary
print("White Pieces")
print(f'{white_pieces:064b}')
print("Black Pieces")
print(f'{black_pieces:064b}')

White Pieces
0000000000000000000000000001000000001000000000000000000000000000
Black Pieces
0000000000000000000000000000100000010000000000000000000000000000


In [12]:
# Generate possible moves
from typing import Tuple

def generate_moves(own, enemy, size) -> Tuple[list, dict]:
    """Generate the possible moves for the current player using bitwise operations"""
    empty = ~(own | enemy)  # Empty squares (not owned by either player)
    unique_moves = []  # List of possible moves
    dir_jump = {}  # Dictionary of moves and the number of pieces that can be captured in each direction

    # Generate moves in all eight directions
    for direction in [N, S, E, W, NW, NE, SW, SE]:
        # We get the pieces that are next to an enemy piece in the direction
        count = 0
        victims = direction(own) & enemy
        if not victims:
            continue

        # We keep getting the pieces that are next to an enemy piece in the direction
        for _ in range(size):
            count += 1
            next_piece = direction(victims) & enemy
            if not next_piece:
                break
            victims |= next_piece

        # We get the pieces that can be captured in the direction
        captures = direction(victims) & empty
        # if there are multiple pieces in captures, we separate them and add them to the set
        while captures:
            capture = captures & -captures  # get the least significant bit
            captures ^= capture  # remove the lsb
            if capture not in dir_jump:
                unique_moves.append(capture)
                dir_jump[capture] = []
            dir_jump[capture].append((direction, count))

    return unique_moves, dir_jump


def make_move(own, enemy, move_to_play, directions, size):
    """Make the move and update the board using bitwise operations."""
    for direction, count in directions[move_to_play]:
        victims = move_to_play  # Init the victims with the move to play

        op_dir = opposite_dir(direction)  # opposite direction since we go from the move to play to the captured pieces
        for _ in range(count):
            victims |= (op_dir(victims) & enemy)
        own ^= victims
        enemy ^= victims & ~move_to_play
    # because of the XOR, the move to play which is considered a victim can be returned a pair number of times
    own |= move_to_play
    return own, enemy


def N(x):
    return (x & 0x00ffffffffffffff) << 8


def S(x):
    return (x & 0xffffffffffffff00) >> 8


def E(x):
    return (x & 0x7f7f7f7f7f7f7f7f) << 1


def W(x):
    return (x & 0xfefefefefefefefe) >> 1


def NW(x):
    return N(W(x))


def NE(x):
    return N(E(x))


def SW(x):
    return S(W(x))


def SE(x):
    return S(E(x))


def opposite_dir(direction):
    if direction == N:
        return S
    if direction == S:
        return N
    if direction == E:
        return W
    if direction == W:
        return E
    if direction == NW:
        return SE
    if direction == NE:
        return SW
    if direction == SW:
        return NE
    if direction == SE:
        return NW

In [13]:
# Generate possible moves for white
print("White Possible Moves")
white_moves, directions = generate_moves(white_pieces, black_pieces, size)
all_w_moves = 0
for move in white_moves:
    all_w_moves |= move
print_pieces(all_w_moves, size)

White Possible Moves
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 1 0 0 0 
0 0 0 0 0 1 0 0 
0 0 1 0 0 0 0 0 
0 0 0 1 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 


In [14]:
# Generate possible moves for black
print("Black Possible Moves")
black_moves, directions = generate_moves(black_pieces, white_pieces, size)
all_b_moves = 0
for move in black_moves:
    all_b_moves |= move
print_pieces(all_b_moves, size)

Black Possible Moves
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 1 0 0 0 0 
0 0 1 0 0 0 0 0 
0 0 0 0 0 1 0 0 
0 0 0 0 1 0 0 0 
0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 


In [15]:
def play(own, enemy, move_to_play, direction, n_jump):
    # The move is guaranteed to be valid, so we don't need to check for it
    victims = direction(own) & enemy            # initial victims
    for _ in range(n_jump - 1):                 # Maximum of n_jump - 1 steps in the direction
        victims |= direction(victims) & enemy   # add to the victims the next opponent piece
    own |= move_to_play | victims
    enemy &= ~victims
    return own, enemy

In [16]:
# Play a move for black
move = black_moves[0]
directions, n_jump = directions[move]
print("Black Move")
print_pieces(move, size)
print("Board")
print_board(white_pieces, black_pieces, size)
black_pieces, white_pieces = play(black_pieces, white_pieces, move, directions, n_jump)
print("White pieces after black move")
print_board(0, black_pieces, size)
print("Black pieces after black move")
print_board(white_pieces, 0, size)
print("Board after black move")
print_board(white_pieces, black_pieces, size)

ValueError: not enough values to unpack (expected 2, got 1)

In [17]:
# Play a move for white
moves, directions = generate_moves(white_pieces, black_pieces)
move = moves[0]
directions, n_jump = directions[move]
print("White Move")
print_pieces(move, size)
print("Board")
print_board(white_pieces, black_pieces, size)
white_pieces, black_pieces = play(white_pieces, black_pieces, move, directions, n_jump)
print("White pieces after white move")
print_board(white_pieces, 0, size)
print("Black pieces after white move")
print_board(0, black_pieces, size)
print("Board after white move")
print_board(white_pieces, black_pieces, size)

TypeError: generate_moves() missing 1 required positional argument: 'size'

In [18]:
# Play a move for black
moves, directions = generate_moves(black_pieces, white_pieces)
move = moves[0]
directions, n_jump = directions[move]
print("Black Move")
print_pieces(move, size)
print("Board")
print_board(white_pieces, black_pieces, size)
black_pieces, white_pieces = play(black_pieces, white_pieces, move, directions, n_jump)
print("White pieces after black move")
print_board(0, black_pieces, size)
print("Black pieces after black move")
print_board(white_pieces, 0, size)
print("Board after black move")
print_board(white_pieces, black_pieces, size)

TypeError: generate_moves() missing 1 required positional argument: 'size'

### Testing a heuristic function

In [19]:
import numpy as np

TABLE1 = np.array([
    500, -150, 30, 10, 10, 30, -150, 500,
    -150, -250, 0, 0, 0, 0, -250, -150,
    30, 0, 1, 2, 2, 1, 0, 30,
    10, 0, 2, 16, 16, 2, 0, 10,
    10, 0, 2, 16, 16, 2, 0, 10,
    30, 0, 1, 2, 2, 1, 0, 30,
    -150, -250, 0, 0, 0, 0, -250, -150,
    500, -150, 30, 10, 10, 30, -150, 500
])

def heuristic(own, enemy, size, table=TABLE1):
    # Convert the binary representations to boolean masks
    own_mask = np.array([bool(own & (1 << i)) for i in range(size*size)])
    enemy_mask = np.array([bool(enemy & (1 << i)) for i in range(size*size)])
    
    # Apply the masks to the table and sum the values
    sum1 = np.sum(table[own_mask])
    sum2 = np.sum(table[enemy_mask])
    
    return sum1 - sum2

own = 0x03_00_00_00_00_00_00_00 # correspond to -150 points following the above table
enemy = 0x00_00_00_00_00_00_00_01 # correspond to 500 points following the above table

result = heuristic(own, enemy, 8)
print(result)


-150


In [20]:
path = "moves.txt"
# read the csv file
average = 0
with open(path, 'r') as file:
    data = file.read()
    data = data.split('\n')
    size_f = len(data)
    for i, line in enumerate(data):
        if line == '':
            continue
        average += int(line)
average /= size_f
print(average)
print(size_f)

FileNotFoundError: [Errno 2] No such file or directory: 'moves.txt'

In [ ]:
white_pieces = 0b00100000_00000000_10100000_11100000_11111011_11111000_10001000_00000000
black_pieces = 0b01000000_10100100_01001111_00011000_00000100_00000010_01100000_01000000

moves, directions = generate_moves(black_pieces, white_pieces, 8)
all_moves = 0
for move in moves:
    all_moves |= move

print_board(white_pieces, black_pieces, 8)
print_pieces(all_moves, size)
print_pieces(white_pieces, size)
print_pieces(black_pieces, size)
print_pieces(white_pieces | black_pieces, size)

In [ ]:
def generate_random_board(size: int):
    white_pieces = 0
    black_pieces = 0
    for i in range(size):
        for j in range(size):
            if number:=np.random.rand() > 0.7:
                white_pieces = set_state(white_pieces, i, j, size)
            elif number > 0.4:
                black_pieces = set_state(black_pieces, i, j, size)
            else:
                # empty cell
                continue
    return white_pieces, black_pieces

In [24]:
black = 16213252669337416725 
white = 6616014524205090956

moves = generate_moves(black, white, 8)[0]
all_move = 0
for move in moves:
    all_move |= move
print(f'{all_move:064b}')
print_pieces(all_move, 8)

0000010000001110001100001001000000000100000011000000000100100010
0 1 0 0 0 1 0 0 
1 0 0 0 0 0 0 0 
0 0 1 1 0 0 0 0 
0 0 1 0 0 0 0 0 
0 0 0 0 1 0 0 1 
0 0 0 0 1 1 0 0 
0 1 1 1 0 0 0 0 
0 0 1 0 0 0 0 0 
